In [2]:
using LinearAlgebra
using Unitful
using UnitfulAstro
using OrdinaryDiffEq
include("./GalaxyCollisionFunctions.jl")

init_disk! (generic function with 2 methods)

6558.124731964162 km s^-1

In [58]:
G = 4.3009E-3 *1u"pc *(km/s)^2 /Msun"
function format_parameters(galaxy_args)
    #I didn't have this at first, but this makes all galaxies have an uniform format. Not too crazy
    return Dict(
        "mass" => galaxy_args[1]*1u"Msun",
        "radius"     => galaxy_args[2]*1u"pc",
        "center_pos" => galaxy_args[3].*1u"pc",
        "center_vel" => galaxy_args[4].*1u"km/s",
        "normal"     => galaxy_args[5],
        "Nᵣ"    => galaxy_args[6],
        "Nₛ"    => galaxy_args[7],
        "softening"  => galaxy_args[8]
    )
end
function init_disk!(galaxy,dT=1E-4u"yr")
    #=
    This function takes a 'galaxy' as an argument, which is an array of 8 arguments=>
        [mass,radius,center_pos,center_vel,normal,N_rings,N_stars,softening]
    And outputs the star positions, velocities and the velocity scale.
    =#
    dr = (1-galaxy["softening"]) * galaxy["radius"]/galaxy["Nᵣ"] # Disregarding the softening, this is just the width of the rings(raidus/number of slices)
    StarsPerRing = trunc(Int64,galaxy["Nₛ"]/galaxy["Nᵣ"])
#-------------------------------------------------------------Rotations--------------------------------------------------------------
    if norm(galaxy["normal"]) == 0 
        Rotation = I
    else 
        cosθ = normalize(galaxy["normal"])[3]
        sinθ = √(1-cosθ^2)
        u = [0,0,1] × normalize(galaxy["normal"])
        u = normalize(u)

        Rotation = [
        u[1]*u[1]*(1-cosθ)+cosθ u[1]*u[2]*(1-cosθ)-u[2]*sinθ u[1]*u[3]*(1-cosθ)+u[1]*sinθ;

        u[2]*u[1]*(1-cosθ)+u[3]*sinθ u[2]*u[2]*(1-cosθ)+cosθ u[2]*u[3]*(1-cosθ)-u[1]*sinθ;

        u[3]*u[1]*(1-cosθ)+u[2]*sinθ u[3]*u[1]*(1-cosθ)+u[1]*sinθ u[3]*u[3]*(1-cosθ)+cosθ
        ]
    end
#-----------------------------------------------------------------------------------------------------------------------------------
    galaxy["star_pos"] = []
    galaxy["star_vel"] = []

    R = galaxy["softening"] * galaxy["radius"]
    for i ∈ 1:galaxy["Nᵣ"] #iterate over the rings
        #Randomly distribute the initial radii and angles 
        rₛ =(R * ones(StarsPerRing) + dr *rand(StarsPerRing))./1u"pc" #creates an n-dimensional vector of radii from R to R+dr 
        ϕₛ = 2π *rand(StarsPerRing)

        # Positions 
        vecᵣ = (Rotation * ([rₛ.*cos.(ϕₛ),rₛ.*sin.(ϕₛ),zeros(StarsPerRing)])).*1u"pc"
        x = ustrip.(u"m",galaxy["center_pos"][1].+vecᵣ[1])#here, we strip our units into our 'integrating' unitlessness because units don't really play nice in arrays of arrays
        y = ustrip.(u"m",galaxy["center_pos"][2].+vecᵣ[2])
        z = ustrip.(u"m",galaxy["center_pos"][3].+vecᵣ[3])

        # Velocities
        Tₛ = 2π * uconvert.(u"s",sqrt.((rₛ*1u"pc").^3/(G *Introoder["mass"])))

        Δϕ = 2π *uconvert(u"s",dT)./Tₛ 

        vecᵥ = (Rotation* [(rₛ/(uconvert(u"s",dT)*1u"1/s")).*(cos.(ϕₛ)-cos.(ϕₛ-Δϕ)),rₛ/(uconvert(u"s",dT)*1u"1/s").*(sin.(ϕₛ)-sin.(ϕₛ-Δϕ)),zeros(StarsPerRing)])*1u"pc/s"
        v₁ = ustrip.(u"m/s",galaxy["center_vel"][1].+vecᵥ[1])
        v₂ = ustrip.(u"m/s",galaxy["center_vel"][2].+vecᵥ[2])
        v₃ = ustrip.(u"m/s",galaxy["center_vel"][3].+vecᵥ[3])
        for j ∈ 1:StarsPerRing
            push!(galaxy["star_pos"],[x[j],y[j],z[j]])
            push!(galaxy["star_vel"],[v₁[j],v₂[j],v₃[j]])
        end
        R += dr
    end 
    galaxy["vel_scale"]=uconvert(u"km/s",√(G*galaxy["mass"]/(0.5*R)))
end


function evolve_disk(galaxy,dT=1e-4u"yr",N_steps=100000,frames=500)
    #Integration stuff
    dT=ustrip(u"s",dT)
    rₘ=ustrip(u"m",galaxy["softening"]*galaxy("radius"))
    Nₛ=galaxy["Nₛ"]
    
    #Galaxy stuff
    M = ustrip(u"Msun",galaxy["mass"])
    R₀ = ustrip.(galaxy["center_pos"])
    V₀ = ustrip.(galaxy["center_vel"])

    #star stuff
    rₛ = galaxy["star_pos"]
    vₛ=galaxy["star_vel"]

    f(dr,r,R,t) = ustrip(u"m^3/(Msun *s^2)",G)*M (R-r)/max(norm(R-r),rₘ)

end

init_disk! (generic function with 2 methods)

In [60]:
Introoder = format_parameters([5e10, 10, [-5,5,1], [15,-15,0], [1,-1,2^0.5], 10, 40, 0.025])
init_disk!(Introoder)

3-element Vector{Float64}:
 -1.4295808562858902e17
  1.386543103439726e17
  2.3899455284736736e16

In [26]:
cosθ = normalize(Introoder["normal"])
Rotation = rand(3,3)
R = Introoder["softening"] * Introoder["radius"]
dr = (1-Introoder["softening"]) * Introoder["radius"]/Introoder["Nᵣ"] # Disregarding the softening, this is just the width of the rings(raidus/number of slices)
StarsPerRing = trunc(Int64,Introoder["Nₛ"]/Introoder["Nᵣ"])
rₛ = (R * ones(StarsPerRing) + dr *rand(StarsPerRing))./1u"pc" #creates an n-dimensional vector of radii from R to R+dr 
ϕₛ = 2π *rand(StarsPerRing)
cosθ = normalize(Introoder["normal"])[3]
        sinθ = √(1-cosθ^2)
        u = [0,0,1] × normalize(Introoder["normal"])
        u = normalize(u)

        Rotation = [
        u[1]*u[1]*(1-cosθ)+cosθ u[1]*u[2]*(1-cosθ)-u[2]*sinθ u[1]*u[3]*(1-cosθ)+u[1]*sinθ;

        u[2]*u[1]*(1-cosθ)+u[3]*sinθ u[2]*u[2]*(1-cosθ)+cosθ u[2]*u[3]*(1-cosθ)-u[1]*sinθ;

        u[3]*u[1]*(1-cosθ)+u[2]*sinθ u[3]*u[1]*(1-cosθ)+u[1]*sinθ u[3]*u[3]*(1-cosθ)+cosθ
        ]
        vecᵣ =Rotation.*[rₛ.*cos.(ϕₛ),rₛ.*sin.(ϕₛ),zeros(StarsPerRing)].*1u"pc"
        Introoder["center_pos"][1] .+ vecᵣ[1]
        #Tₛ = 2π *sqrt.(((rₛ*1u"pc")^3)/(G *galaxy["mass"]))
   uconvert.(u"s",sqrt.((rₛ*1u"pc").^3/(G *Introoder["mass"])))
   dT=1E-4u"yr"
uconvert(u"s",dT)



evolve_disk (generic function with 4 methods)